In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch

import pyro
import pyro.infer
import pyro.optim
import pyro.distributions as dist
from IPython.display import Markdown, Math

pyro.set_rng_seed(101)

In [2]:
def model():
    # off, on
    A = pyro.sample("A", dist.Categorical(torch.tensor([0.5, 0.5])))
    prob_B = torch.tensor([
        [0.8, 0.2], # A = off
        [0.1, 0.9] # A = on
    ])
    B = pyro.sample("B", dist.Categorical(prob_B[A]))
    prob_C = torch.tensor([
        # B = off
        [
            [0.9, 0.1], # A = off
            [0.99, 0.01] # A = on
        ],
        # B = on
        [
            [0.1, 0.9], # A = off
            [0.4, 0.6] # A = on
        ]
    ])
    C = pyro.sample("C", dist.Categorical(prob_C[B, A]))

In [3]:
model_cond = pyro.condition(model, data={"B": torch.tensor(1), "C": torch.tensor(1)})

In [4]:
imp_cond = pyro.infer.Importance(
    model=model_cond, num_samples=20000)

inf_cond = imp_cond.run()

In [16]:
p_a_on_given_b_c_on = torch.exp(inf_cond.marginal("A").empirical["A"].log_prob(1))

In [17]:
Math(f"P(A=on|B=on, C=on) = {p_a_on_given_b_c_on:.3}")

<IPython.core.display.Math object>

In [10]:
model_do = pyro.do(
    pyro.condition(model, data={"C": torch.tensor(1)}),
    data={"B": torch.tensor(1)}
)

In [11]:
imp_do = pyro.infer.Importance(
    model=model_do, num_samples=10000)

In [12]:
inf_do = imp_do.run()

In [13]:
p_a_on_given_c_on_do_b_on = torch.exp(inf_do.marginal("A").empirical["A"].log_prob(1))

In [14]:
Math(f"P(A=on|do(B=on), C=on) = {p_a_on_given_c_on_do_b_on:.3}")

<IPython.core.display.Math object>